In [1]:
import numpy as np
from src.NN import NN
import src.utils as utils

# Settings
csv_filename = "data/seeds_dataset.csv"
hidden_layers = [1,2] # number of nodes in hidden layers i.e. [layer1, layer2, ...]
eta = 0.1 # learning rate
n_epochs = 50 # number of training epochs
n_folds = 4 # number of folds for cross-validation
seed_crossval = 1 # seed for cross-validation
seed_weights = 1 # 

In [2]:

print("Reading '{}'...".format(csv_filename))
X, y, n_classes = utils.read_csv(csv_filename, target_name="y", normalize=True)

Reading 'data/seeds_dataset.csv'...


In [3]:
m,n=X.shape
n

7

In [4]:
def get_splits(X):
    datasets,features=X.shape

    splitWeights=list()
    splits = np.hsplit(X,features)
    # return splits

    for o,oX in enumerate(splits):
        N, d = oX.shape

        print("Neural network model:")
        print(" input_dim = {}".format(d))
        print(" hidden_layers = {}".format(hidden_layers))
        print(" output_dim = {}".format(n_classes))
        print(" eta = {}".format(eta))
        print(" n_epochs = {}".format(n_epochs))
        print(" n_folds = {}".format(n_folds))
        print(" seed_crossval = {}".format(seed_crossval))
        print(" seed_weights = {}\n".format(seed_weights))

        # Create cross-validation folds
        idx_all = np.arange(0, N)
        idx_folds = utils.crossval_folds(N, n_folds, seed=seed_crossval) # list of list of fold indices

        # Train/evaluate the model on each fold
        acc_train, acc_valid = list(), list()
        print("Cross-validating with {} folds...".format(len(idx_folds)))
        for i, idx_valid in enumerate(idx_folds):
#  seed=seed_weights
            # Collect training and test data from folds
            idx_train = np.delete(idx_all, idx_valid)
            X_train, y_train = oX[idx_train], y[idx_train]
            X_valid, y_valid = oX[idx_valid], y[idx_valid]

            # Build neural network classifier model and train
            model = NN(input_dim=d, output_dim=n_classes,
                        hidden_layers=hidden_layers)
            model.train(X_train, y_train, eta=eta, n_epochs=n_epochs)

            # Make predictions for training and test data
            ypred_train = model.predict(X_train)
            ypred_valid = model.predict(X_valid)

            # Compute training/test accuracy score from predicted values
            acc_train.append(100*np.sum(y_train==ypred_train)/len(y_train))
            acc_valid.append(100*np.sum(y_valid==ypred_valid)/len(y_valid))

            # Print cross-validation result
            print(" Fold {}/{}: acc_train = {:.2f}%, acc_valid = {:.2f}% (n_train = {}, n_valid {})".format(i+1,n_folds, acc_train[-1], acc_valid[-1], len(X_train), len(X_valid)))

        # Print results
        print(o," : ")
        print("  -> acc_train_avg = {:.2f}%, acc_valid_avg = {:.2f}%".format(sum(acc_train)/float(len(acc_train)), sum(acc_valid)/float(len(acc_valid))))

        splitWeights.append(model.get_weights())
        
    
    return splitWeights

In [6]:
wt = get_splits(X)

55393286035514] [0.2056161259223287] ***^^ 

[1.606867282207214] [0.2056161259223287] ***^^ 

[-0.7356684651611319, -0.9133720838036807] [0.5020846235625482, 0.5818561707497238] ***^^ 

[-1.2575266409445582, 0.5955795230644011] [0.5020846235625482, 0.5818561707497238] ***^^ 

[-0.5903369500430965, -0.7767408983753207] [0.5020846235625482, 0.5818561707497238] ***^^ 

[1.7271965968317686] [-0.74377857] ***^^ 

[0.04055393286035514] [0.2167594216762383] ***^^ 

[1.606867282207214] [0.2167594216762383] ***^^ 

[-0.7356684651611319, -0.9133720838036807] [0.5021975976073391, 0.5862061515108987] ***^^ 

[-1.2575266409445582, 0.5955795230644011] [0.5021975976073391, 0.5862061515108987] ***^^ 

[-0.5903369500430965, -0.7767408983753207] [0.5021975976073391, 0.5862061515108987] ***^^ 

[1.7271965968317686] [-1.47771663] ***^^ 

[0.04055393286035514] [0.07227164243648732] ***^^ 

[1.606867282207214] [0.07227164243648732] ***^^ 

[-0.7356684651611319, -0.9133720838036807] [0.5007327243092504, 0.52

In [7]:
(wt [1][1])len(new[1])

SyntaxError: invalid syntax (<ipython-input-7-64ffa6b64d4b>, line 1)

In [17]:
# len(model.get_weights()[0][])
datasets,features=X.shape

In [18]:
new=[]
import copy
print("wth",new)
rep = copy.deepcopy(wt)
for i,mx in enumerate(rep):
    # m2 = wt[i]
    if i==0:
        print("hitting new")
        new = mx
        print(new[1])
    else:
        for j,lx in enumerate(mx):
            if j==0 and i!=0:
                # print(new[j],"88",lx)
                
                new[j] = np.append(new[j],lx)
                print(len(new[j]),"*******************")
            else:
                new[j] = np.add(new[j],lx)
                print(len(new[j]))

for k in range(len(new)):
    if not k == 0:
        print(k)
        new[k] = np.divide(new[k],len(wt))
        # print(type(l1))
print( new )

wth []
hitting new
[[2.1209449227289645], [-0.19948450982853366]]
2 *******************
2
3
3 *******************
2
3
4 *******************
2
3
5 *******************
2
3
6 *******************
2
3
7 *******************
2
3
1
2
[array([ 2.16220611,  2.51110376,  1.09143156,  1.29916867,  1.97122   ,
       -0.6693636 ,  1.7271966 ]), array([[0.64490089],
       [1.27604509]]), array([[-0.71725832, -0.86275405],
       [-0.63578179,  0.01427466],
       [-0.50197709, -0.78656383]])]


In [19]:
new_nn =  NN(input_dim=features, output_dim=n_classes,
                        hidden_layers=hidden_layers,AvgWeights=new)

In [20]:
new_nn.get_weights()

node :  {'weights': [2.162206105066135, 2.511103755489059, 1.0914315598966, 1.299168673146784, 1.9712199964449098, -0.6693635968466675, 1.7271965968317686], 'output': None, 'delta': None}
node :  {'weights': [[0.6449008887629116], [1.2760450946757522]], 'output': None, 'delta': None}
node :  {'weights': [[0.6449008887629116], [1.2760450946757522]], 'output': None, 'delta': None}
node :  {'weights': [[-0.7172583226205111, -0.862754050663147], [-0.635781787018038, 0.01427466090778091], [-0.5019770862022626, -0.7865638296664071]], 'output': None, 'delta': None}
node :  {'weights': [[-0.7172583226205111, -0.862754050663147], [-0.635781787018038, 0.01427466090778091], [-0.5019770862022626, -0.7865638296664071]], 'output': None, 'delta': None}
node :  {'weights': [[-0.7172583226205111, -0.862754050663147], [-0.635781787018038, 0.01427466090778091], [-0.5019770862022626, -0.7865638296664071]], 'output': None, 'delta': None}


[[[2.162206105066135,
   2.511103755489059,
   1.0914315598966,
   1.299168673146784,
   1.9712199964449098,
   -0.6693635968466675,
   1.7271965968317686]],
 [[[0.6449008887629116], [1.2760450946757522]],
  [[0.6449008887629116], [1.2760450946757522]]],
 [[[-0.7172583226205111, -0.862754050663147],
   [-0.635781787018038, 0.01427466090778091],
   [-0.5019770862022626, -0.7865638296664071]],
  [[-0.7172583226205111, -0.862754050663147],
   [-0.635781787018038, 0.01427466090778091],
   [-0.5019770862022626, -0.7865638296664071]],
  [[-0.7172583226205111, -0.862754050663147],
   [-0.635781787018038, 0.01427466090778091],
   [-0.5019770862022626, -0.7865638296664071]]]]

In [26]:
nw = [[[2.162206105066135,
   2.511103755489059,
   1.0914315598966,
   1.299168673146784,
   1.9712199964449098,
   -0.6693635968466675,
   1.7271965968317686]],
 [[[0.6449008887629116], [1.2760450946757522]],
  [[0.6449008887629116], [1.2760450946757522]]],
 [[[-0.7172583226205111, -0.862754050663147],
   [-0.635781787018038, 0.01427466090778091],
   [-0.5019770862022626, -0.7865638296664071]],
  [[-0.7172583226205111, -0.862754050663147],
   [-0.635781787018038, 0.01427466090778091],
   [-0.5019770862022626, -0.7865638296664071]],
  [[-0.7172583226205111, -0.862754050663147],
   [-0.635781787018038, 0.01427466090778091],
   [-0.5019770862022626, -0.7865638296664071]]]]

In [27]:
new [1]

array([[0.64490089],
       [1.27604509]])

In [21]:
# for oX in enumerate(splits):
N, d = X.shape

print("Neural network model:")
print(" input_dim = {}".format(d))
print(" hidden_layers = {}".format(hidden_layers))
print(" output_dim = {}".format(n_classes))
print(" eta = {}".format(eta))
print(" n_epochs = {}".format(n_epochs))
print(" n_folds = {}".format(n_folds))
print(" seed_crossval = {}".format(seed_crossval))
print(" seed_weights = {}\n".format(seed_weights))

# Create cross-validation folds
idx_all = np.arange(0, N)
idx_folds = utils.crossval_folds(N, n_folds, seed=seed_crossval) # list of list of fold indices

# Train/evaluate the model on each fold
acc_train, acc_valid = list(), list()
print("Cross-validating with {} folds...".format(len(idx_folds)))
for i, idx_valid in enumerate(idx_folds):
#  seed=seed_weights
    # Collect training and test data from folds
    idx_train = np.delete(idx_all, idx_valid)
    X_train, y_train = X[idx_train], y[idx_train]
    X_valid, y_valid = X[idx_valid], y[idx_valid]

    # Build neural network classifier model and train
    # model = NN(input_dim=d, output_dim=n_classes,
    #             hidden_layers=hidden_layers)
    new_nn.train(X_train, y_train, eta=eta, n_epochs=n_epochs)

    # Make predictions for training and test data
    ypred_train = new_nn.predict(X_train)
    ypred_valid = new_nn.predict(X_valid)

    # Compute training/test accuracy score from predicted values
    acc_train.append(100*np.sum(y_train==ypred_train)/len(y_train))
    acc_valid.append(100*np.sum(y_valid==ypred_valid)/len(y_valid))

    # Print cross-validation result
    print(" Fold {}/{}: acc_train = {:.2f}%, acc_valid = {:.2f}% (n_train = {}, n_valid {})".format(i+1,n_folds, acc_train[-1], acc_valid[-1], len(X_train), len(X_valid)))

# Print results
print(o," : ")
print("  -> acc_train_avg = {:.2f}%, acc_valid_avg = {:.2f}%".format(sum(acc_train)/float(len(acc_train)), sum(acc_valid)/float(len(acc_valid))))

Neural network model:
 input_dim = 7
 hidden_layers = [1, 2]
 output_dim = 3
 eta = 0.1
 n_epochs = 50
 n_folds = 4
 seed_crossval = 1
 seed_weights = 1

Cross-validating with 4 folds...
[2.162206105066135, 2.511103755489059, 1.0914315598966, 1.299168673146784, 1.9712199964449098, -0.6693635968466675, 1.7271965968317686] [ 0.08766787  0.16281232 -0.05732034  0.25848062  0.07986931 -0.92262221
 -0.42370003] ***^^ 

[[0.6449008887629116], [1.2760450946757522]] [0.7139895040470035] ***^^ 



TypeError: float() argument must be a string or a number, not 'list'

In [ ]:
new=[]
import copy
print("wth",new)
rep = copy.deepcopy(wt)
for i,mx in enumerate(rep):
    # m2 = wt[i]
    if i==0:
        print("hitting new")
        new = mx
        print(new[1])
    else:
        for j,lx in enumerate(mx):
            if j==0 and i!=0:
                # print(new[j],"88",lx)
                
                new[j] = np.append(new[j],lx)
                print(len(new[j]),"*******************")
            else:
                new[j] = np.add(new[j],lx)
                print(len(new[j]))

for k in range(len(new)):
    if not k == 0:
        print(k)
        new[k] = np.divide(new[k],len(wt))
        # print(type(l1))
print( new )

In [ ]:
rp=0
for fp in range(len(wt)):
    print(wt[fp][1])
    rp += wt[fp][1][0][0]
    print(rp)
print(rp/7)

In [ ]:
np.asarray([ 3.9273942 ,  1.9839786 ,  1.7497002 ,  3.1951442 ,  1.8707918 ,
       -0.18531519,  1.9123502 ], dtype="float32")

In [29]:
wt[1][1]

[[1.185179945930981], [2.1572901168471024]]

In [ ]:
for()